In [2]:
from nltk.util import ngrams
from collections import Counter

# 샘플 텍스트
text = "자연어 처리는 재미있다."

# 2-gram 생성
tokens = text.split()
bigrams = list(ngrams(tokens, 2))
bigram_freq = Counter(bigrams)

print("2-gram:", bigrams)
print("2-gram 빈도:", bigram_freq)

2-gram: [('자연어', '처리는'), ('처리는', '재미있다.')]
2-gram 빈도: Counter({('자연어', '처리는'): 1, ('처리는', '재미있다.'): 1})


In [3]:
import math  # 수학 연산(로그, 거듭제곱) 사용

# 예제 확률값
probs = [0.1, 0.2, 0.3, 0.4]  # 모델이 각 토큰에 할당한 확률들
perplexity = math.pow(2, -sum(math.log2(p) for p in probs) / len(probs))  # 평균 로그확률의 음수를 2의 거듭제곱으로 바꿔 퍼플렉서티 계산
print("퍼플렉서티:", perplexity)  # 계산된 퍼플렉서티 출력


퍼플렉서티: 4.518010018049224


In [4]:
import torch  # PyTorch 기본 라이브러리
import torch.nn as nn  # 신경망 레이어 모음

class NNLM(nn.Module):  # 간단한 Neural Network Language Model
    def __init__(self, vocab_size, embed_size, hidden_size):
        super(NNLM, self).__init__()  # nn.Module 초기화
        self.embed = nn.Embedding(vocab_size, embed_size)  # 단어 ID → 임베딩 벡터
        self.fc1 = nn.Linear(embed_size, hidden_size)  # 임베딩 → 은닉공간 변환
        self.fc2 = nn.Linear(hidden_size, vocab_size)  # 은닉공간 → 단어 분포(logits)

    def forward(self, x):
        x = self.embed(x)  # 입력 단어를 벡터로 변환
        x = torch.relu(self.fc1(x))  # 비선형 변환으로 특징 추출
        x = self.fc2(x)  # 각 단어에 대한 점수 계산
        return x  # softmax 전 logits 반환

model = NNLM(vocab_size=5000, embed_size=300, hidden_size=128)  # 모델 생성
print(model)  # 모델 구조 출력


NNLM(
  (embed): Embedding(5000, 300)
  (fc1): Linear(in_features=300, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=5000, bias=True)
)


In [5]:
class Seq2Seq(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Seq2Seq, self).__init__()
        # 인코더 LSTM : input -> hidden
        self.encoder = nn.LSTM(input_size, hidden_size, batch_first=True)
        # 디코더 LSTM : hidden -> output
        self.decoder = nn.LSTM(hidden_size, output_size, batch_first=True)

    def forward(self, x):
        _, (hidden, _) = self.encoder(x)  # 입력을 인코딩하여 hidden 추출
        out, _ = self.decoder(hidden)     # hidden을 디코더 입력으로 넣어 out 추출
        return out